In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd 
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC


In [2]:
data = pd.read_csv('final_with_covid_AS.csv')
data.head()

,CountCases,BedDys,BeddystoCC,Severity,DiedCases,DiedBdDys,Hsptlsns,April,August,December,...,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021,CovidPrd
0,17,133,7.823529,75,4,9,13,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,13,372,28.615385,63,4,34,9,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,106,13.250000,48,4,13,4,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,45,218,4.844444,153,3,24,42,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,24,238,9.916667,90,3,8,21,0,1,0,...,1,0,0,0,0,0,0,0,0,0


In [3]:
filtered_data = data[data['CarDriver'] == 1]
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12582 entries, 11 to 54025
Data columns (total 58 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CountCases       12582 non-null  int64  
 1   BedDys           12582 non-null  int64  
 2   BeddystoCC       12582 non-null  float64
 3   Severity         12582 non-null  int64  
 4   DiedCases        12582 non-null  int64  
 5   DiedBdDys        12582 non-null  int64  
 6   Hsptlsns         12582 non-null  int64  
 7   April            12582 non-null  int64  
 8   August           12582 non-null  int64  
 9   December         12582 non-null  int64  
 10  February         12582 non-null  int64  
 11  January          12582 non-null  int64  
 12  July             12582 non-null  int64  
 13  June             12582 non-null  int64  
 14  March            12582 non-null  int64  
 15  May              12582 non-null  int64  
 16  November         12582 non-null  int64  
 17  October    

In [4]:
num_bins = 3

bed_days_min = filtered_data['Severity'].min()
bed_days_max = filtered_data['Severity'].max()

bin_width = (bed_days_max - bed_days_min) / num_bins
bins = [bed_days_min + i * bin_width for i in range(num_bins + 1)]

def bin_function(x):
    for i in range(1, len(bins)):
        if x <= bins[i]:
            return i - 1
    return i

filtered_data['Bed_days_category'] = filtered_data['Severity'].apply(bin_function)

/var/folders/3l/xh28kn0j7_7d5hh7fgz188940000gn/T/ipykernel_3715/3168803887.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Bed_days_category'] = filtered_data['Severity'].apply(bin_function)


In [6]:
filtered_data['Bed_days_category'].value_counts()

0    11983
1      495
2      104
Name: Bed_days_category, dtype: int64

In [38]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
from imblearn.over_sampling import RandomOverSampler

# تقسیم‌بندی داده‌ها به ورودی و خروجی

X = filtered_data[['Age1725','Age2639','Age4064','Age6574','Age75plus',
                   'Male',
                   
                   'Regional',

                   'April','December','February','January','July',
                   'June','March','May','November','October','September',

                   'Cntrprt23WldMV','CntrprtFxdStnry','CntrprtHvyVhcl',
                   'CntrprtNClsn','CntrprtOther','CntrprtNMV','CntrprtCrTrkVn',
                   'CntrprtPdlC','CntrprtPdstAnml','CntrprtTrn',

                   'Y2014','Y2015','Y2016','Y2017','Y2018','Y2019','Y2020','Y2021',
                   'Y2013','Y2012',
                   'CovidPrd']]

y = filtered_data['Bed_days_category']

#ros = RandomOverSampler(random_state=42)

#X_resampled, y_resampled = ros.fit_resample(X, y)

# سپس ادامه کد با داده‌های جدید
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
# ایجاد و آموزش مدل XGBoost
model = XGBClassifier(num_class=3, random_state=42)

model.fit(X_train, y_train)


# ارزیابی مدل
y_pred_xgb = model.predict(X_test)
acc_xgb = accuracy_score(y_test, y_pred_xgb)
conf_matrix_xgb = confusion_matrix(y_test, y_pred_xgb)
class_report_xgb = classification_report(y_test, y_pred_xgb)

precision_xgb = precision_score(y_test, y_pred_xgb, average='macro') 
recall_xgb = recall_score(y_test, y_pred_xgb, average='macro')
f1_xgb = f1_score(y_test, y_pred_xgb, average='macro')

print('Accuracy:', acc_xgb)
print('Confusion Matrix:\n', conf_matrix_xgb)
print('Classification Report:\n', class_report_xgb)
print('Precision:', precision_xgb)
print('Recall:', recall_xgb)
print('F1-score:', f1_xgb)

Accuracy: 0.9859602649006622
Confusion Matrix:
 [[3589   18    0]
 [  17  116    9]
 [   0    9   17]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      3607
           1       0.81      0.82      0.81       142
           2       0.65      0.65      0.65        26

    accuracy                           0.99      3775
   macro avg       0.82      0.82      0.82      3775
weighted avg       0.99      0.99      0.99      3775

Precision: 0.820106866695885
Recall: 0.821919088550482
F1-score: 0.8210096305472344


In [20]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTEENN, SMOTETomek
from sklearn.datasets import make_classification


oversampler = SMOTE(random_state=42,sampling_strategy='not majority')
X_smote, y_smote = oversampler.fit_resample(X_train, y_train)

oversampler = ADASYN(random_state=42,sampling_strategy='not majority')
X_adasyn, y_adasyn = oversampler.fit_resample(X_smote, y_smote)


In [30]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
from imblearn.over_sampling import RandomOverSampler

# تقسیم‌بندی داده‌ها به ورودی و خروجی

X = filtered_data[['Age1725','Age2639','Age4064','Age6574','Age75plus',
                   'Male',
                   
                   'Regional',

                   'April','December','February','January','July',
                   'June','March','May','November','October','September',

                   'Cntrprt23WldMV','CntrprtFxdStnry','CntrprtHvyVhcl',
                   'CntrprtNClsn','CntrprtOther','CntrprtNMV','CntrprtCrTrkVn',
                   'CntrprtPdlC','CntrprtPdstAnml','CntrprtTrn',

                   'Y2014','Y2015','Y2016','Y2017','Y2018','Y2019','Y2020','Y2021',
                   'Y2013','Y2012',
                   'CovidPrd']]

y = filtered_data['Bed_days_category']

ros = RandomOverSampler(random_state=42,sampling_strategy='minority')

#X_resampled, y_resampled = ros.fit_resample(X, y)

# سپس ادامه کد با داده‌های جدید
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
# ایجاد و آموزش مدل XGBoost
model = XGBClassifier(objective='multi:softprob' , num_class=3, random_state=42)

model.fit(X_resampled, y_resampled)




# ارزیابی مدل
y_pred_xgb = model.predict(X_test)
acc_xgb = accuracy_score(y_test, y_pred_xgb)
conf_matrix_xgb = confusion_matrix(y_test, y_pred_xgb)
class_report_xgb = classification_report(y_test, y_pred_xgb)

precision_xgb = precision_score(y_test, y_pred_xgb, average='macro') 
recall_xgb = recall_score(y_test, y_pred_xgb, average='macro')
f1_xgb = f1_score(y_test, y_pred_xgb, average='macro')

print('Accuracy:', acc_xgb)
print('Confusion Matrix:\n', conf_matrix_xgb)
print('Classification Report:\n', class_report_xgb)
print('Precision:', precision_xgb)
print('Recall:', recall_xgb)
print('F1-score:', f1_xgb)

Accuracy: 0.9854304635761589
Confusion Matrix:
 [[3589   18    0]
 [  17  113   12]
 [   0    8   18]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      3607
           1       0.81      0.80      0.80       142
           2       0.60      0.69      0.64        26

    accuracy                           0.99      3775
   macro avg       0.80      0.83      0.81      3775
weighted avg       0.99      0.99      0.99      3775

Precision: 0.8027450917801532
Recall: 0.8276973478498683
F1-score: 0.8140917518556153


In [31]:
filtered_data5 = data[data['Pedestrian'] == 1]
filtered_data5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6086 entries, 0 to 54037
Data columns (total 58 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CountCases       6086 non-null   int64  
 1   BedDys           6086 non-null   int64  
 2   BeddystoCC       6086 non-null   float64
 3   Severity         6086 non-null   int64  
 4   DiedCases        6086 non-null   int64  
 5   DiedBdDys        6086 non-null   int64  
 6   Hsptlsns         6086 non-null   int64  
 7   April            6086 non-null   int64  
 8   August           6086 non-null   int64  
 9   December         6086 non-null   int64  
 10  February         6086 non-null   int64  
 11  January          6086 non-null   int64  
 12  July             6086 non-null   int64  
 13  June             6086 non-null   int64  
 14  March            6086 non-null   int64  
 15  May              6086 non-null   int64  
 16  November         6086 non-null   int64  
 17  October      

In [43]:
num_bins = 3

bed_days_min = filtered_data5['Severity'].min()
bed_days_max = filtered_data5['Severity'].max()

bin_width = (bed_days_max - bed_days_min) / num_bins
bins = [bed_days_min + i * bin_width for i in range(num_bins + 1)]

def bin_function(x):
    for i in range(1, len(bins)):
        if x <= bins[i]:
            return i - 1
    return i

filtered_data5['Bed_days_category'] = filtered_data5['Severity'].apply(bin_function)

/var/folders/3l/xh28kn0j7_7d5hh7fgz188940000gn/T/ipykernel_3715/1308772640.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data5['Bed_days_category'] = filtered_data5['Severity'].apply(bin_function)


In [44]:
filtered_data5['Bed_days_category'].value_counts()

0    5521
1     532
2      33
Name: Bed_days_category, dtype: int64

In [53]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTEENN, SMOTETomek
from sklearn.datasets import make_classification


oversampler = SMOTE(random_state=42,sampling_strategy='not majority')
X_smote, y_smote = oversampler.fit_resample(X_train, y_train)

oversampler = ADASYN(random_state=42,sampling_strategy='not majority')
X_adasyn, y_adasyn = oversampler.fit_resample(X_smote, y_smote)

In [54]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
from imblearn.over_sampling import RandomOverSampler

# تقسیم‌بندی داده‌ها به ورودی و خروجی

X = filtered_data5[['Age1725','Age2639','Age4064','Age6574','Age75plus',
                   'Male',
                   
                   'Regional',

                   'April','December','February','January','July',
                   'June','March','May','November','October','September',

                   'Cntrprt23WldMV','CntrprtFxdStnry','CntrprtHvyVhcl',
                   'CntrprtNClsn','CntrprtOther','CntrprtNMV','CntrprtCrTrkVn',
                   'CntrprtPdlC','CntrprtPdstAnml','CntrprtTrn',

                   'Y2014','Y2015','Y2016','Y2017','Y2018','Y2019','Y2020','Y2021',
                   'Y2013','Y2012',
                   'CovidPrd']]

y = filtered_data5['Bed_days_category']

ros = RandomOverSampler(random_state=42)

X_resampled, y_resampled = ros.fit_resample(X, y)

# سپس ادامه کد با داده‌های جدید
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#

In [55]:
#X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
# ایجاد و آموزش مدل XGBoost
model = XGBClassifier(num_class=3, random_state=42)

model.fit(X_adasyn, y_adasyn)


# ارزیابی مدل
y_pred_xgb = model.predict(X_test)
acc_xgb = accuracy_score(y_test, y_pred_xgb)
conf_matrix_xgb = confusion_matrix(y_test, y_pred_xgb)
class_report_xgb = classification_report(y_test, y_pred_xgb)

precision_xgb = precision_score(y_test, y_pred_xgb, average='macro') 
recall_xgb = recall_score(y_test, y_pred_xgb, average='macro')
f1_xgb = f1_score(y_test, y_pred_xgb, average='macro')

print('Accuracy:', acc_xgb)
print('Confusion Matrix:\n', conf_matrix_xgb)
print('Classification Report:\n', class_report_xgb)
print('Precision:', precision_xgb)
print('Recall:', recall_xgb)
print('F1-score:', f1_xgb)

Accuracy: 0.8817086527929902
Confusion Matrix:
 [[1495  138    4]
 [  57  113    9]
 [   1    7    2]]
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.91      0.94      1637
           1       0.44      0.63      0.52       179
           2       0.13      0.20      0.16        10

    accuracy                           0.88      1826
   macro avg       0.51      0.58      0.54      1826
weighted avg       0.91      0.88      0.89      1826

Precision: 0.511323586423543
Recall: 0.5815136240727405
F1-score: 0.5381555155436636
